from: https://langchain-ai.github.io/langgraph/tutorials/chatbot-simulation-evaluation/agent-simulation-evaluation/



In [11]:
import getpass
import os
from langchain_core.tools import tool
from langchain_deepseek import ChatDeepSeek

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("DEEPSEEK_API_KEY")

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=10, 
)

from typing import List

# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    messages = [system_message] + messages
    completion = llm.invoke(messages)
    return completion.model_dump()

my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': 'Hello! Welcome to [Airline Name] customer support. How can I assist you today? 😊  \n\nAre you looking for help with:  \n- **Flight bookings**  \n- **Check-in or boarding passes**  \n- **Baggage policies**  \n- **Flight status**  \n- **Changes or cancellations**  \n\nLet me know, and I’ll be happy to help! ✈️',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 82,
   'prompt_tokens': 15,
   'total_tokens': 97,
   'completion_tokens_details': None,
   'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0},
   'prompt_cache_hit_tokens': 0,
   'prompt_cache_miss_tokens': 15},
  'model_name': 'deepseek-chat',
  'system_fingerprint': 'fp_3d5141a69a_prod0225',
  'id': '857f842d-813a-4e4f-a4d4-8746097138c8',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-0adb0f4f-6e69-4eba-acdd-aa71dc1f5894-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Harrison. You are trying to get a refund for the trip you took to Alaska. \
You want them to give you ALL the money back. \
This trip happened 5 years ago."""

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = llm

simulated_user = prompt | model

In [13]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hi! How can I help you?")]
simulated_user.invoke({"messages": messages})

AIMessage(content='Hi there. My name is Harrison. I need a full refund for my trip to Alaska that I took five years ago. The service was terrible and I want all my money back.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 91, 'total_tokens': 128, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 91}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '161c5e2f-494a-4f51-b544-7a036fd56118', 'finish_reason': 'stop', 'logprobs': None}, id='run-43445a36-e327-4e34-b3be-4851e1a4c637-0', usage_metadata={'input_tokens': 91, 'output_tokens': 37, 'total_tokens': 128, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [14]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # Convert from LangChain format to the OpenAI format, which our chatbot function expects.
    messages = [convert_message_to_dict(m) for m in messages]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

In [15]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return {"messages": [HumanMessage(content=response.content)]}

In [16]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

In [17]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

In [18]:
for chunk in simulation.stream({"messages": []}):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': {'messages': [AIMessage(content="Hello! Thank you for reaching out. How can I assist you today? Whether you have questions about flights, bookings, baggage, or any other travel-related concerns, I'm here to help. Please let me know the details of your inquiry, and I’ll do my best to provide a quick and accurate solution.  \n\nFor faster assistance, you may also provide:  \n- Your booking reference/ticket number  \n- Flight details (date, route, airline)  \n- Specific issue or request  \n\nLooking forward to assisting you! ✈️", additional_kwargs={}, response_metadata={}, id='926db145-452e-4ef6-90ca-b6d47084b317')]}}
----
{'user': {'messages': [HumanMessage(content='Hi there. My name is Harrison. I need a full refund for my trip to Alaska that I took five years ago. The service was terrible and I want ALL my money back. No partial refunds - I want every single penny returned to me immediately. \n\nI don\'t have the booking details anymore because it was so long ago, but I kn